The two lines below require the MongoDB Java client as well as a library called [Fongo](https://github.com/fakemongo/fongo) which lets us play around with MongoDB without actually needing a MongoDB server running.
Fongo should **never** be used for any serious use case as the performance will

In [1]:
%maven org.mongodb:mongodb-driver:3.5.0
%maven com.github.fakemongo:fongo:2.1.0

Before we do anything else, we need to create a connection to the server.

In [2]:
import com.github.fakemongo.Fongo;
import com.mongodb.MongoClient;

// Create a connection to our (fake) MongoDB instance
// In a *real* program, we would instead use new MongoClient(host, port);
MongoClient mongoClient = new Fongo("test").getMongo();

Let's insert some data so we have something to play with.
Don't worry about the details right now.
We're just connecting to the database `mydb` and inserting the contents of [`zips.json`](zips.json) into the `zips` collection.

In [3]:
import com.mongodb.DBObject;
import org.bson.types.BasicBSONList;

String zipsJson = new Scanner(new File("zips.json")).useDelimiter("\\Z").next();
BasicBSONList obj = (BasicBSONList) com.mongodb.util.JSON.parse(zipsJson);
mongoClient.getDB("mydb").getCollection("zips").insert((DBObject[]) obj.toArray(new DBObject[0]));

WriteResult{n=149, updateOfExisting=false, upsertedId=null}

Now that we have some documents to work with, let's write some queries.
First we get a reference to the database and then the specific collection within the database.
The variable `printBlock` will simply be used to display the results (more on blocks later).

In [4]:
import com.mongodb.Block;
import com.mongodb.client.MongoCollection;
import com.mongodb.client.MongoDatabase;

import org.bson.Document;

MongoDatabase database = mongoClient.getDatabase("mydb");
MongoCollection<Document> collection = database.getCollection("zips");

Block<Document> printBlock = new Block<Document>() {
       @Override
       public void apply(final Document document) {
           System.out.println(document.toJson());
       }
};

The simplest query we can execute is to retrieve all the documents in our database which are printed by `printBlock`.

In [5]:
collection.find().forEach(printBlock);

{ "_id" : "01701", "city" : "FRAMINGHAM", "loc" : [-71.425486, 42.300665], "pop" : 65046, "state" : "MA" }
{ "_id" : "02154", "city" : "NORTH WALTHAM", "loc" : [-71.236497, 42.382492], "pop" : 57871, "state" : "MA" }
{ "_id" : "02401", "city" : "BROCKTON", "loc" : [-71.034348, 42.081571], "pop" : 59498, "state" : "MA" }
{ "_id" : "02840", "city" : "MIDDLETOWN", "loc" : [-71.30348, 41.504502], "pop" : 47687, "state" : "RI" }
{ "_id" : "02860", "city" : "PAWTUCKET", "loc" : [-71.390713, 41.872873], "pop" : 45442, "state" : "RI" }
{ "_id" : "02895", "city" : "NORTH SMITHFIELD", "loc" : [-71.513683, 41.99948], "pop" : 53733, "state" : "RI" }
{ "_id" : "03060", "city" : "NASHUA", "loc" : [-71.466684, 42.756395], "pop" : 41438, "state" : "NH" }
{ "_id" : "03103", "city" : "MANCHESTER", "loc" : [-71.449325, 42.965563], "pop" : 36613, "state" : "NH" }
{ "_id" : "03301", "city" : "CONCORD", "loc" : [-71.527734, 43.218525], "pop" : 34035, "state" : "NH" }
{ "_id" : "04240", "city" : "LEWISTON", 

{ "_id" : "55106", "city" : "SAINT PAUL", "loc" : [-93.048817, 44.968384], "pop" : 47905, "state" : "MN" }
{ "_id" : "55112", "city" : "NEW BRIGHTON", "loc" : [-93.199691, 45.074129], "pop" : 44128, "state" : "MN" }
{ "_id" : "55337", "city" : "BURNSVILLE", "loc" : [-93.275283, 44.76086], "pop" : 51421, "state" : "MN" }
{ "_id" : "57103", "city" : "SIOUX FALLS", "loc" : [-96.686415, 43.537386], "pop" : 32508, "state" : "SD" }
{ "_id" : "57401", "city" : "ABERDEEN", "loc" : [-98.485642, 45.466109], "pop" : 28786, "state" : "SD" }
{ "_id" : "57701", "city" : "ROCKERVILLE", "loc" : [-103.200259, 44.077041], "pop" : 45328, "state" : "SD" }
{ "_id" : "58103", "city" : "FARGO", "loc" : [-96.812252, 46.856406], "pop" : 38483, "state" : "ND" }
{ "_id" : "58501", "city" : "BISMARCK", "loc" : [-100.774755, 46.823448], "pop" : 36602, "state" : "ND" }
{ "_id" : "58701", "city" : "MINOT", "loc" : [-101.298476, 48.22914], "pop" : 42195, "state" : "ND" }
{ "_id" : "59102", "city" : "BILLINGS", "loc" 

Now that we know what our data looks like, we can write some more interesting queries.
In Java, we use a similar syntax to what we would use in the MongoDB shell.
See the [MongoDB read documentation](https://mongodb.github.io/mongo-java-driver/3.9/driver/tutorials/perform-read-operations/) for more details.

In [6]:
import com.mongodb.client.model.Filters;
import com.mongodb.client.model.Projections;

import static com.mongodb.client.model.Filters.and;
import static com.mongodb.client.model.Filters.gte;
import static com.mongodb.client.model.Filters.eq;
import static com.mongodb.client.model.Projections.fields;
import static com.mongodb.client.model.Projections.include;

// The code below is equivalent to the following query in the MongoDB shell
// db.zips.find({$and: [{state: {$eq: "OR"}}, {pop: {$gte: 44000}}]}, {city: 1})
collection.find(and(eq("state", "OR"), gte("pop", 44000)))
          .projection(fields(include("city")))
          .forEach(printBlock);

{ "_id" : "97005", "city" : "BEAVERTON" }
{ "_id" : "97301", "city" : "SALEM" }


That's it for this basic overview of MongoDB in Java.
You should now be able to get started writing your own programs which interact with a database using this as a starting point.
If you're just viewing the static output of this tutorial, go to [the repository](https://github.com/michaelmior/db-notebooks) and click the launch button.
This will allow you to edit all the code interactively and try modifying it as you wish.